<a href="https://colab.research.google.com/github/ipeirotis/dealing_with_data/blob/master/02-WebAPIs/A2-NewsAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re
from bs4 import BeautifulSoup

In [ ]:
# Note, sign up and get your own key for NewsAPI

news_api_key = 'KEY ON SLACK'

In [ ]:
import requests

In [ ]:
endpoint = 'https://newsapi.org/v2/top-headlines'
parameters = {
    'country' : 'us',
    'category' : 'business',
    'apiKey' : news_api_key,
    'pageSize' : 100
}
resp = requests.get(endpoint, params=parameters)

In [ ]:
data = resp.json()

In [ ]:
data.keys()

In [ ]:
data['status']

In [ ]:
data['totalResults']

In [ ]:
# This is  a list of the returned articles
data['articles']

In [ ]:
# Do not freak out. It is just a big list.
# This is the first article in the list
data['articles'][0]

## Exercise

Examine the documentation of the [`top-headlines` endpoint](https://newsapi.org/docs/endpoints/top-headlines). Add the ability to search for a specific keyword. Make it a function that gets as input the keyword and returns back the articles.

In [ ]:
# Your code here

### Analyzing the returned JSON objects that represent the articles

In [ ]:
# This is the first article in the list
data['articles'][0]

In [ ]:
# Let's see the keys for an individual article
data['articles'][0].keys()

In [ ]:
data['articles'][0]['description']

In [ ]:
data['articles'][0]['content']

### Iterate over all articles and do various things

In [ ]:
# That is a convenient way to create a datafame
# from a **list** of **JSON/dictionaries**. A bit awkward
# though if we want to work with each individual article

df = pd.json_normalize( data['articles'] )
df

In [ ]:
# Print the titles of the news articles
for article in data['articles']:
    print(article['title'])
    print('-------')

### Exercise

Modify the code above to print also the URL of each article.

In [ ]:
# YOUR CODE HERE

### Exercise

Modify the code above to print also the URL of each article, and print the length of the title.

### Example

Create a dataframe  the code above to print also the URL of each article, and print the length of the title. Create a histogram of the title lengths.

In [ ]:
results = [] # We store here our results

# We go through all the articles
for article in data['articles']:
  # Remember that "article" is the loop variable

  # We create our own dictionaries, with the attributes that we need
  entry = {
      "title": article['title'],
      "url": article['url'],
      "title_length": len(article['title']) # Length of the title in characters
  }
  results.append(entry)

df = pd.DataFrame(results)
df

In [ ]:
# Create a histogram of title lengths
df['title_length'].hist(figsize=(6,2), bins=20)

In [ ]:
# Create one big string with all the titles
titles = '\n'.join(df['title'])
print(titles)

### Exercise

We have a helper functions below. Use the `retrieve_text_from_url` to retrieve the actual content of the URL and store in the dataframe.

In [ ]:
# Helper function: This functions gets as input a URL
# and returns back the text that appears in that web page
def retrieve_text_from_url(url):
    """Remove html tags from a string"""
    try:
      resp = requests.get(url)
      soup = BeautifulSoup(resp.text, "html.parser")
      return soup.get_text()
    except:
      return ""

In [ ]:
results = [] # We store here our results

# We go through all the
for article in data['articles']:
  c = article['content']

  # START CODE
  url_content = ... # YOUR CODE HERE
  # END CODE
  entry = {
      "title": article['title'],
      "url": article['url'],
      "description": article['description'],
      "content": article['content'],
      "url_content": ... # YOUR CODE HERE
  }
  results.append(entry)

df_content = pd.DataFrame(results)

## Exercises


1. Go through all the results, and detect the language of the returned articles. Store the results in a dataframe.

2. Concatenate all titles into one big string.

3. Concatenate all descriptions into one big string.